In [6]:
#import libraries

import pandas as pd
import prophet as Prophet
import yfinance as yf
import plotly.graph_objects as go

import warnings

from datetime import datetime, timedelta


In [7]:
warnings.filterwarnings("ignore")
pd.options.display.float_format = "${:,.2f}".format